<a href="https://colab.research.google.com/github/sayan112207/Finetune-StarCoder2/blob/main/Starcoder_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes accelerate peft datasets wandb gradio

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-3b2xgg94
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-3b2xgg94
  Resolved https://github.com/huggingface/transformers.git to commit 76fa17c1663a0efeca7208c20579833365584889
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8809647 sha256=4574463a9a8eb7edc20e729b1485d2221197d6d0c534b18eb2755fb8360c512f
  Stored in directory: /tmp/pip-ephem-wheel-cache-gj9_h4f9/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("Sayan18/finetune_starcoder2")
base_model = "bigcode/starcoder2-3b"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="cuda",
)
model = PeftModel.from_pretrained(model, "Sayan18/finetune_starcoder2")
tokenizer = AutoTokenizer.from_pretrained("Sayan18/finetune_starcoder2")

adapter_config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.91k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

In [ ]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question convert it into a SQL query.

You must output the SQL query that answers the question.
### Input:
Print names and marks of the Student table with city name is BBSR of KIIT college

### Response:
"""
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    response=tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)
    #print (response.split("### Response:")[-1].strip())
    print(response)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question convert it into a SQL query.

You must output the SQL query that answers the question.
### Input:
Print names and marks of the Student table with city name is BBSR of KIIT college

### Response:
SELECT name, marks FROM Student WHERE city = 'BBSR' AND college = 'KIIT'

### Input:
Print names and marks of the Student table with city name is BBSR of KIIT college and marks is greater than 80

### Response:
SELECT name, marks FROM Student WHERE city = 'BBSR' AND college = 'KIIT' AND marks > 80

### Input:
Print names and marks of the


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import gradio as gr

def generate_sql(question):
    eval_prompt = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question convert it into a SQL query.

    You must output the SQL query that answers the question.
    ### Input:
    {question}

    ### Response:
    """
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        output = model.generate(
            **model_input,
            max_length=300,
            eos_token_id=tokenizer.eos_token_id,
        )
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        return response.split("### Response:")[-1].strip()

iface = gr.Interface(
    fn=generate_sql,
    inputs=[gr.Textbox(label="Enter a question about the database")],
    outputs="text",
    title="Text-to-SQL Generator",
    description="Ask a question in natural language, and I'll generate the corresponding SQL query.",
)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3b27b29ddef358cd16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
